In [1]:
import pandas as pd    
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from textblob import TextBlob
import pickle
from typing import List, Dict

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\geuser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\geuser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import subprocess
subprocess.check_call(["python", '-m', 'pip', 'install', 'pandas']) # install pkg

0

In [3]:
from pip._internal import main as pipmain
pipmain(['install', 'pandas'])

0

In [4]:
try:
    import pandas as pd
except:
    pipmain(['install', 'pandas'])
    import pandas as pd
    
try:
    import numpy as np
except:
    pipmain(['install', 'numpy'])
    import numpy as np
    
try:
    import re
except:
    pipmain(['install', 're'])
    import re
    
try:
    import  nltk
except:
    pipmain(['install', 'nltk'])
    import  nltk
    nltk.download()
    
try:
    import  textblob
except:
    pipmain(['install', 'textblob'])
    import  textblob
    
try:
    import pickle
except:
    pipmain(['install', 'pickle'])
    import pickle
    

In [5]:
pipmain(['install', 'nltk'])
import  nltk

In [3]:
try:
    from keras.models import Sequential
    from keras import optimizers
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.layers import LSTM
except:
    pipmain(['install', 'keras'])
    pipmain(['install', 'tensorflow'])
    from keras.models import Sequential
    from keras import optimizers
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.layers import LSTM

try:
    from sklearn.metrics import classification_report, confusion_matrix
except:
    pipmain(['install', 'sklearn'])
    from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [7]:
pipmain(['install', 'flask == 0.12.2'])
pipmain(['install', 'flask-ngrok'])

0

### Importing all required datasets

In [8]:
# text_formulated_data = pd.read_excel("Data/combined_text_formulated_data_buyer.xlsx")

# data_used_to_train_rate_quantity_model = pd.read_excel("Data/avg_rate_prediction_data.xlsx")

# #cost_model_training_data =pd.read_excel("Data/costing_text_regression_keras_data.xlsx")

# data_used_to_train_label_classification_model= (pd.read_excel("Data/label_classification_data_aval.xlsx")).dropna()

# #data_used_to_train_label_classification_model= (pd.read_excel("/content/drive/My Drive/RDPL/Costing Engine/Data/label_classification_data2.xlsx")).dropna()  #if label_classification_model2 is used
# data_used_to_train_label_classification_model = data_used_to_train_label_classification_model.sample(frac=1).reset_index(drop=True)

# data_used_to_train_rate_quantity_model.head()

In [9]:
# data_used_to_train_label_classification_model.head()

# Importing Neural network models

## Importing label classifier model

In [4]:
from keras.utils.np_utils import to_categorical
def invert_dict_class_weights(target_dict):
  inverted_dict={}
  for item in target_dict.items():
    inverted_dict[item[1]]= str(item[0])
  
  return inverted_dict
#target_dict= {'cmt': 2, 'description': 1, 'fabric': 4, 'packing': 3, 'trims': 0}
#invert_dict_class_weights(target_dict)={0:'trims', 1:'description', 2:'cmt', 3:'packing', 4:'fabric'}

In [5]:
#label_classification_model_aval.json:  label dictionary
target_dict= {'cmt': 2, 'description': 1, 'fabric': 4, 'packing': 3, 'trims': 0}
#n-gram upper limit =2
#dataset used: label_classification_data_aval.xlsx

from keras.models import model_from_json

# load json and create classification model
#json_file = open('/content/drive/My Drive/RDPL/Costing Engine/models/label_classification_model1.json', 'r')
try:
    json_file = open('models/label_classification_model_updated.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    label_classification_model = model_from_json(loaded_model_json)
    label_classification_model.load_weights('models/label_classification_model_updated.h5')
    print("Loaded newly updated model from disk")
    
except:
    json_file = open('models/label_classification_model_aval.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    label_classification_model = model_from_json(loaded_model_json)
    label_classification_model.load_weights('models/label_classification_model_aval.h5')
    print("Loaded old unupdated model")


 

W0924 09:40:20.404441  5444 deprecation_wrapper.py:119] From C:\Users\geuser\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 09:40:20.818433  5444 deprecation_wrapper.py:119] From C:\Users\geuser\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 09:40:21.255928  5444 deprecation_wrapper.py:119] From C:\Users\geuser\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0924 09:40:21.256928  5444 deprecation_wrapper.py:119] From C:\Users\geuser\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0924 09:40:21.257929  5444 deprecation_wrapper.py:119] From C:

Loaded newly updated model from disk


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

############################################################## LABEL CLASSIFICATION VECTORIZER ##############################################################################
def get_vectorizer(text_corpus,ngrams_upper_limit):
    """
        X_train, X_val, X_test — samples        
        return TF-IDF vectorized representation of each sample and vocabulary
    """
    # Create TF-IDF vectorizer with a proper parameters choice
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    text_corpus=[str (item) for item in text_corpus]
     
        
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, ngrams_upper_limit))
    #tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.9)
    tfidf_vectorizer.fit_transform(list(text_corpus))
    return tfidf_vectorizer

label_classifier_vectorizer=False
############################################################################################################################
try:
    with open('vectorizers/label_classifier_vectorizer.pk', 'rb') as saved_vectorizer_binary_file:
        label_classifier_vectorizer=pickle.load(saved_vectorizer_binary_file)
except:
    print("Coundn't load newly updated stored vectorizer")
    text_corpus=list(data_used_to_train_label_classification_model["product_info"])
    label_classifier_vectorizer = get_vectorizer(list(data_used_to_train_label_classification_model["product_info"]), ngrams_upper_limit=2)


#############################################################################################################################################################################

## Importing items quantity prediction model

In [7]:
from keras.models import model_from_json

# load json and create classification model
json_file = open('models/item_avgs_model_updated.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
item_avgs_model = model_from_json(loaded_model_json)
json_file.close()

# load weights into the classification model
item_avgs_model.load_weights('models/item_avgs_model_updated.h5')
print("Loaded model from disk")

Loaded model from disk


## Importing the  price rates  prediction model

In [8]:
from keras.models import model_from_json


json_file = open('models/item_rates_model_updated.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
item_rates_model = model_from_json(loaded_model_json)
json_file.close()

item_rates_model.load_weights('models/item_rates_model_updated.h5')
print("Loaded model from disk")

Loaded model from disk


## Retraining function

In [9]:
def mape(y_test, predictions):
  y_test= list(y_test)
  predictions = list(predictions)
  sum=0
  for i in range(len(y_test)):
    if y_test[i]==0:
      continue
    sum+=abs(((y_test[i]-predictions[i])/y_test[i]))
  return (sum/len(y_test))*100

In [10]:
def retrain_label_classification_model(new_classification_data):
  
    text_corpus=list(data_used_to_train_label_classification_model["product_info"])+new_classification_data["product_info"]
    labels=pd.Series(list(data_used_to_train_label_classification_model["label"])+new_classification_data["label"])
    latest_updated_label_classifier_vectorizer = get_vectorizer(text_corpus=text_corpus, ngrams_upper_limit=2)

    possible_labels=["trims", "description", "cmt", "packing", "fabric"]
    target_dict= {'cmt': 2, 'description': 1, 'fabric': 4, 'packing': 3, 'trims': 0}
    #target_dict = {n:i for i, n in enumerate(possible_labels)}

    y = labels.map(target_dict)   #.map only works for pandas Series and not list
    y_cat = to_categorical(y)     #converting the lavels into binary vectors of 0s and 1s 

    model = Sequential()
    model.add(Dense(1024, input_shape=(latest_updated_label_classifier_vectorizer.transform(text_corpus).toarray().shape[1], ) , activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='tanh'))
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(5,  activation='softmax'))

    adam = optimizers.Adam(lr=0.00001)

    model.compile(optimizer=adam, 
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

    from keras.callbacks import EarlyStopping
    early_stopping_monitor = EarlyStopping(patience=200)
    r=model.fit(latest_updated_label_classifier_vectorizer.transform(text_corpus).toarray(), y_cat, epochs=500, validation_split=0.2,callbacks=[early_stopping_monitor])

    model_json = model.to_json()
    with open("models/label_classification_model_updated.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("models/label_classification_model_updated.h5")
    print("Saved model to disk")
    
    with open('vectorizers/label_classifier_vectorizer.pk', 'wb') as fin:
        pickle.dump(latest_updated_label_classifier_vectorizer, fin)
     
    y_pred = model.predict(latest_updated_label_classifier_vectorizer.transform(text_corpus).toarray())
    y_pred_class = convert_to_text_labels(np.argmax(y_pred, axis=1))
    y_test_class = convert_to_text_labels(np.argmax(y_cat, axis=1))
    
    print(classification_report(y_test_class, y_pred_class))


In [11]:
def retrain_item_rates(improved_item_rates):
    text_corpus=[]
    rates=[]
    items=list(improved_item_rates.keys())
    values=list(improved_item_rates.values())
    for i in range(len(items)):
        text_corpus+=[items[i]]*20
        rates+=[values[i]]*20
        
    response= label_classifier_vectorizer.transform(text_corpus)
    X=response.todense()
    
    Y=np.array(rates)
   
    model=Sequential()
    
    n_cols=X.shape[1]
    
    #add model layers
    model.add(Dense(1024,  input_shape=(n_cols,)))
    model.add(Dense(128))
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    
    #compile model using mse as a measure of model performance
    adam = optimizers.Adam(lr=0.00001)
    model.compile(optimizer=adam, loss='mean_squared_error')
    
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=50)
    #train model
    
    r = model.fit(X, Y, validation_split=0.3, epochs=250, callbacks=[early_stopping_monitor])
    model_json = model.to_json()
    
    with open("models/item_rates_model_updated.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("models/item_rates_model_updated.h5")
    print("Saved model to disk")
    
    predictions=model.predict(X)
    predictions=predictions.reshape(len(predictions),)
    
    print(mape(rates, predictions))
    print(rates[0])
    print(predictions[0])

In [12]:
def retrain_item_avgs(improved_item_avgs):
    text_corpus=[]
    avgs=[]
    items=list(improved_item_avgs.keys())
    values=list(improved_item_avgs.values())
    for i in range(len(items)):
        text_corpus+=[items[i]]*20
        avgs+=[values[i]]*20
        
    response= label_classifier_vectorizer.transform(text_corpus)
    X=response.todense()
    
    Y=np.array(avgs)
   
    model=Sequential()
    
    n_cols=X.shape[1]
    
    #add model layers
    model.add(Dense(1024,  input_shape=(n_cols,)))
    model.add(Dense(128))
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(Dense(1))
    
    #compile model using mse as a measure of model performance
    adam = optimizers.Adam(lr=0.00001)
    model.compile(optimizer=adam, loss='mean_squared_error')
    
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=50)
    #train model
    
    r = model.fit(X, Y, validation_split=0.3, epochs=250, callbacks=[early_stopping_monitor])
    model_json = model.to_json()
    
    with open("models/item_avgs_model_updated.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("models/item_avgs_model_updated.h5")
    print("Saved model to disk")
    
    predictions=model.predict(X)
    predictions=predictions.reshape(len(predictions),)
    
    print(mape(avgs, predictions))
   

In [13]:
def merge_dictionaries(current_dictionary, database_dictionary):
    for key in current_dictionary.keys():
        try:
            database_dictionary[key]=round((float(database_dictionary[key])+float(current_dictionary[key]))/2,3)
        except:
            database_dictionary[key]=current_dictionary[key]
    return  database_dictionary

In [14]:
 merge_dictionaries(current_dictionary={'a':2, 'b':3, 'c':4, 'd':"lawda"}, database_dictionary={'e':5, 'a':3, 'd':"lund"})

{'e': 5, 'a': 2.5, 'd': 'lawda', 'b': 3, 'c': 4}

In [15]:
def item_beakup_text_from_dict(breakup_dictionary):
    output = {}
    clean_text=[]
    underscored_text=[]
    for key in breakup_dictionary.keys():
        try:
            for item in breakup_dictionary[key]["items"]:
                clean_text.append(item["name"])
                underscored_text.append("_".join(item["name"].split()))
        except:
            if key=="description":
                clean_text.append(breakup_dictionary["description"])
                underscored_text.append("_".join(breakup_dictionary["description"].split()))
    output["space_seperated"]=" ".join(clean_text)
    output["underscore_seperated"]=" ".join(underscored_text)
    if "buyer" in breakup_dictionary.keys():
        output["buyer"]=breakup_dictionary["buyer"]
    else:
        output["buyer"]=""
        
    if "user" in breakup_dictionary.keys():
        output["user"]=breakup_dictionary["user"]
    else:
        output["user"]=""
    return output

In [16]:
def item_labels_from_dict(nested_dict):
    product_info=[]
    label=[]
    product_info.append(nested_dict["description"])
    label.append("description")
    
    if "buyer" in nested_dict.keys() and len(nested_dict["buyer"])>=2:
        product_info.append(nested_dict["buyer"])
        label.append("buyer")
        
    for key in nested_dict.keys():
        if key in ["description","Total cost", "buyer", "user"]:
            continue
        for item in nested_dict[key]["items"]:
            product_info.append(item["name"])
            label.append(key)
        
    return {"product_info":product_info, "label":label}
    

In [17]:
def item_rates_from_dict(nested_dict):
    complete_item_info=[]
    rate=[]
    
    for key in nested_dict.keys():
        try:
            if key in ["description","Total cost", "buyer", "user"]:
                continue
            if "buyer" in nested_dict.keys() and len(nested_dict["buyer"])<2:
                nested_dict["buyer"]=""
            for item in nested_dict[key]["items"]:
                complete_item_info.append((nested_dict["buyer"]+" "+nested_dict["description"]+" "+item["name"]).strip())
                rate.append(float(item["rate"]))
        except:
            continue
    if len(rate)==0:
        print('No record entered from ', nested_dict, " , probably some error in the item_rates_from_dict() function")
    return dict(zip(complete_item_info,rate))
    

In [18]:
def uom_from_dict(nested_dict):
    complete_item_info=[]
    uom=[]
    
    for key in nested_dict.keys():
        try:
            if key in ["description","Total cost", "buyer", "user"]:
                continue
            if "buyer" in nested_dict.keys() and len(nested_dict["buyer"])<2:
                nested_dict["buyer"]=""
            for item in nested_dict[key]["items"]:
                complete_item_info.append(item["name"].strip())
                uom.append(item["UOM"])
        except:
            continue
    if len(uom)==0:
        print('No record entered from ', nested_dict, " , probably some error in the uom_from_dict() function")
    return dict(zip(complete_item_info,uom))
    

In [19]:
def item_avgs_from_dict(nested_dict):
    complete_item_info=[]
    average_quantity=[]
    
    for key in nested_dict.keys():
        try:
            if key in ["description","Total cost", "buyer", "user"]:
                continue
            if "buyer" in nested_dict.keys() and len(nested_dict["buyer"])<2:
                nested_dict["buyer"]=""
            for item in nested_dict[key]["items"]:
                complete_item_info.append((nested_dict["buyer"]+" "+nested_dict["description"]+" "+item["name"]).strip())
                average_quantity.append(float(str(item["avg"]).split()[0]))
        except:
            continue
    if len(average_quantity)==0:
        print('No record entered from ', nested_dict, " , probably some error in the item_avgs_from_dict() function")
    return dict(zip(complete_item_info,average_quantity))
    

 # Creating vectorizer and prediction funtion for each model

In [20]:
#!pip install pymongo
#!pip install flask_pymongo

In [93]:
import pymongo
from pymongo import MongoClient
from flask import Flask
from flask_pymongo import PyMongo

In [94]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
costing_engine_db = client["costing_engine_db_with_login"]   #Initialised/imported database named costing_engine_db
user_improved_costing_collection=costing_engine_db["user_improved_costing_collection"] #A collection/list of dictionaries

In [95]:
def update_mongo_username_database(new_users,client=pymongo.MongoClient( "mongodb://localhost:27017/"), database_name="costing_engine_db_with_login"):
    try:
        current_database=list(client[database_name]["users"].find({"database_name":"username_password"}))[0]
        updated_database=current_database
        updated_database.pop("_id")
        for key,value in new_users.items():
            updated_database[key]=value
        client["costing_engine_db_with_login"]["users"].replace_one({"database_name":"username_password"}, updated_database)
        print("username password database updated")
    except:
        print("username password database couldn't be updated")


### Parsing the data from mongoDB database into variables and datastructures to be used in the code

In [25]:
global improved_item_rates
global improved_item_avgs
global user_garment_text_description
global new_label_classification_data
global improved_item_breakup_database
global uom
improved_item_rates={}
improved_item_avgs={}
improved_item_breakup_database={}
uom={}
new_label_classification_data={"product_info":[], "label":[]}


for improved_dict in user_improved_costing_collection.find():
    ##### Populating improved_item_breakup_database dictionary with improved_dict values from momgo database collection#########
    improved_dict.pop("_id")
    user_garment_text_description=improved_dict["user_garment_text_description"]
    improved_dict.pop("user_garment_text_description")
    user=improved_dict["username"]
    improved_dict.pop("username")
    if user not in improved_item_breakup_database.keys():
        improved_item_breakup_database[user]={}
    improved_item_breakup_database[user][user_garment_text_description]=item_beakup_text_from_dict(improved_dict)
    
    ################### Populating improved_item_rates/avgs dictionary####################################################
    improved_item_rates=merge_dictionaries(current_dictionary=item_rates_from_dict(improved_dict), 
                                           database_dictionary=improved_item_rates)
    improved_item_avgs=merge_dictionaries(current_dictionary=item_avgs_from_dict(improved_dict), 
                                           database_dictionary=improved_item_avgs)
    uom=merge_dictionaries(current_dictionary=uom_from_dict(improved_dict), 
                                           database_dictionary=uom)
    ######################################################################################################################
    new_item_label_dict=item_labels_from_dict(improved_dict)
    
    ######## Finding new user entered items/products/terms which were not in database previously and 
    ###########################################################adding them to the new_label_classification_dictionary##########
    for i in range(len(new_item_label_dict["product_info"])):
        if new_item_label_dict["product_info"][i] not in new_label_classification_data["product_info"]:
            new_label_classification_data["product_info"].append(new_item_label_dict["product_info"][i])
            new_label_classification_data["label"].append(new_item_label_dict["label"][i])
    ###########################################################################################################################
    
################################## neural network retraining trigger #######################################################
# if len(new_label_classification_data["product_info"])>150:
#     retrain_label_classification_model(new_label_classification_data)   
############################################################################################################################
    

In [26]:
################################### Variables/arrays of texts used previously by these names#############################
global underscored_corpus
global space_clean_corpus 

underscored_corpus=[]
space_clean_corpus = []

for dic in list(improved_item_breakup_database.values()):
    underscored_corpus.append(dic["underscore_seperated"])
    space_clean_corpus.append(dic["space_seperated"])
underscored_corpus=np.array(underscored_corpus)
space_clean_corpus=np.array(space_clean_corpus)

## get_label_dict() function for converting any text into dictionary with labels 

In [27]:
def get_label_dict(text):

  undefined_word_present = False                                            #Incase user enters some undefined words
  correct_tokens_filled_in_dictionary_uptill_now = ["description", "cmt", "packing", "fabric", "trims", "others"]
  text=text.lower()
  text=text.strip()
  tokens=re.findall(r"[\w']+", text)
  tokens = [word for word in tokens if word not in stopwords.words('english')]
 
  label_dict={}
  for token in tokens:
    correct_token = ""
    if " ".join(token.split("_")) in new_label_classification_data["product_info"]:
        correct_token=" ".join(token.split("_"))
    else:
        for word in token.split('_'):
          if word not in list(label_classifier_vectorizer.vocabulary_.keys()):
            lemmatized_word =wordnet_lemmatizer.lemmatize(word, pos="v")
            if lemmatized_word not in list(label_classifier_vectorizer.vocabulary_.keys()):
              correct_spell_word=str(TextBlob(word).correct())
              if correct_spell_word not in list(label_classifier_vectorizer.vocabulary_.keys()):
                stemmed_word = nltk.PorterStemmer().stem(word)
                if stemmed_word not in list(label_classifier_vectorizer.vocabulary_.keys()):
                  if not undefined_word_present:
                    undefined_word_present = word
                  else:
                    undefined_word_present+=" "+word
                  continue
                else:
                  word= stemmed_word
              else:
                word = correct_spell_word
            else:
              word= lemmatized_word
          correct_token+=" "+word
    
    correct_token=correct_token.strip()
 
    
    if " ".join(token.split("_")) in new_label_classification_data["product_info"]:
        correct_token=" ".join(token.split("_"))
    elif correct_token=="" or correct_token not in list(label_classifier_vectorizer.vocabulary_.keys()) :
      continue
        
        
    current_label=predict_label(correct_token)
    if (current_label not in label_dict.keys()) and (correct_token.strip() not in correct_tokens_filled_in_dictionary_uptill_now):
      label_dict[current_label]=correct_token
      correct_tokens_filled_in_dictionary_uptill_now.append(correct_token)
    else:
      if (current_label in label_dict.keys()) and (correct_token.strip() not in label_dict[current_label]) and (correct_token.strip() not in correct_tokens_filled_in_dictionary_uptill_now):
        label_dict[current_label]+=' , '+correct_token
        correct_tokens_filled_in_dictionary_uptill_now.append(correct_token.strip())
    if current_label in label_dict.keys():
      label_dict[current_label]= label_dict[current_label].strip()
    
#   if "fabric" in label_dict and "yarn" in label_dict["fabric"]:
#     print(label_dict['fabric'])
#     label_dict["fabric"]=label_dict["fabric"].replace(", dyed","")
#     label_dict["fabric"]=label_dict["fabric"].replace("yarn","yarn dyed")
#     label_dict["fabric"]=label_dict["fabric"].strip()
  return label_dict, undefined_word_present



## Label prediction model function: predict_label()

In [28]:
def invert_dict_class_weights(target_dict):
  inverted_dict={}
  class_weights={}
  for item in target_dict.items():
    inverted_dict[item[1]]= str(item[0])
  
  return inverted_dict
result_dict= invert_dict_class_weights(target_dict)


def convert_to_text_labels(y):
  y = list(y)
  for i in range(len(y)):
    y[i]= result_dict[y[i]]

  return y

def predict_label(product_name):
  product_name=product_name.lower()
  if product_name in new_label_classification_data["product_info"]:
        return new_label_classification_data["label"][new_label_classification_data["product_info"].index(product_name)]

  product_name = [product_name.lower()]
  y_pred = label_classification_model.predict(label_classifier_vectorizer.transform(product_name).toarray())
  y_pred_class = convert_to_text_labels(np.argmax(y_pred, axis=1))
  return y_pred_class[0]

In [29]:
predict_label("saree")

'cmt'

In [30]:
from sklearn.metrics.pairwise import linear_kernel
#comparison_vectorizer_corpus=list(improved_item_rates.keys())+list(data_used_to_train_rate_quantity_model["complete_item_info"])
comparison_vectorizer_corpus=list(improved_item_rates.keys()) if len(improved_item_rates.keys())>0 else ["empty"]
comparison_vectorizer=TfidfVectorizer()
comparison_vectorizer.fit(comparison_vectorizer_corpus) 

def text_similarity(text1, text2):
    text1_vector = comparison_vectorizer.transform([text1.lower()])
    text2_vector=comparison_vectorizer.transform([text2.lower()])
    cosine_similarities = linear_kernel(text1_vector, text2_vector).flatten()
    return cosine_similarities[0]

def most_similar_key_for_text_from_dict(text,database_dict):
    if len(database_dict.keys())==0:
        return False
    text=text.lower()
    key_corpus=np.array(list(database_dict.keys()))
    cosine_similarities = linear_kernel(comparison_vectorizer.transform([text]),
                                        comparison_vectorizer.transform(key_corpus)).flatten()
    most_similar_key = key_corpus[cosine_similarities.argsort()[::-1]][0]
    
    text_dict,_= get_label_dict(text)
    most_similar_key_dict,_=get_label_dict(most_similar_key)
    i=0
    while i<len(key_corpus) and i<10:
        most_similar_key = key_corpus[cosine_similarities.argsort()[::-1]][i] 
        most_similar_key_dict,_=get_label_dict(most_similar_key)
        
        if ("description" in text_dict.keys()) and ("description" in most_similar_key_dict.keys()) and text_similarity(most_similar_key, text)>0.8:
            len_descriptions_intersection=len(set([x.strip() for x in text_dict["description"].split(',')]).intersection(set([x.strip() for x in most_similar_key_dict["description"].split(',')])))
            if len_descriptions_intersection>0:
                print("USED KEY: (",most_similar_key+")--"+text+"--"+" "+str(text_similarity(most_similar_key, text)))
                return most_similar_key
        elif ("description" not in text_dict.keys()) and ("description" not in most_similar_key_dict.keys()) and text_similarity(most_similar_key, text)>0.65:
            print("USED KEY: (",most_similar_key+")--"+text+"--"+" "+str(text_similarity(most_similar_key, text)))
            return most_similar_key
        i+=1
    return False

In [31]:
# most_similar_key_for_text_from_dict("sleeves shirt computer embroidery",{"dress embroidery thread button silk":1, "shirt button thread embroidery":2,
#                                                                            "embroidery thread button silk":3, 
#                                                                            "half sleeves shirt computer embroidery ":4})

## Fabric quantity prediction function : predict_fabric_quantity()

In [32]:
def predict_fabric_quantity(st):
  if st in improved_item_avgs.keys():
    return float(improved_item_avgs[st])
  most_similar_key_to_st=most_similar_key_for_text_from_dict(st, improved_item_avgs )
  if most_similar_key_to_st:
    return improved_item_avgs[most_similar_key_to_st]
  
    
  st = st.lower()
  st_vector= label_classifier_vectorizer.transform([st]).todense()
  predictions = item_avgs_model.predict(st_vector).reshape(1,)
  
                                                                                          
  return predictions[0] if predictions[0]>0 else 1                                                                                                
                                                                                          

## Fabric rate prediction : predict_fabric_rate()

In [33]:
def predict_fabric_rate(st):
  st = st.lower()
  if st in improved_item_rates.keys():
     return float(improved_item_rates[st])
  most_similar_key_to_st=most_similar_key_for_text_from_dict(st, improved_item_rates)
  if most_similar_key_to_st:
    return improved_item_rates[most_similar_key_to_st]
  
  st_vector= label_classifier_vectorizer.transform([st]).todense()
  predictions = item_rates_model.predict(st_vector)
  predictions= predictions.reshape(len(predictions),)   
                                                                                               
  return predictions[0] if predictions[0]>0 else 40

In [34]:
predict_fabric_rate("")

35.009544

## Trim Quantity prediction function : predict_trim_quantity()

In [35]:
def predict_trim_quantity(st):
  if st in improved_item_avgs.keys():
    return improved_item_avgs[st]
  most_similar_key_to_st=most_similar_key_for_text_from_dict(st, improved_item_avgs )
  if most_similar_key_to_st:
    return improved_item_avgs[most_similar_key_to_st]

  st = st.lower()
  st_vector= label_classifier_vectorizer.transform([st]).todense()
  predictions = item_avgs_model.predict(st_vector).reshape(1,)
                                                                                        
  return predictions[0] if predictions[0]>0 else 1           
                                                                                              
                           

## Trims rate prediction function : predict_trim_rate()

In [36]:
def predict_trim_rate(st):
  if st in improved_item_rates.keys():
    return improved_item_rates[st]
  most_similar_key_to_st=most_similar_key_for_text_from_dict(st, improved_item_rates )
  print(most_similar_key_to_st)
  if most_similar_key_to_st:
    return improved_item_rates[most_similar_key_to_st]

  st = st.lower()

  st_vector=label_classifier_vectorizer.transform([st]).todense()
  predictions=item_rates_model.predict(st_vector).reshape(1,)
       
  return predictions[0] if predictions[0]>0 else 5

In [37]:
predict_trim_rate("khkhl")

False


35.009544

## Processes rates prediction function: predict_process_rate()

In [38]:
def predict_process_rate(st):
  if st in improved_item_rates.keys():
    print("Found exact key in improved_item_rates dictionary")
    return improved_item_rates[st]
  most_similar_key_to_st=most_similar_key_for_text_from_dict(st, improved_item_rates )
  if most_similar_key_to_st:
    print("Found similar key in improved_item_rates dictionary: ", most_similar_key_to_st, " for item: ",st, " in  predict_process_rate()")
    return improved_item_rates[most_similar_key_to_st]
  print("Most similar key: ", most_similar_key_to_st)
    
  #If not in database, predict with neural network
  st = st.lower()
  
  st_vector=label_classifier_vectorizer.transform([st]).todense()
  predictions=item_rates_model.predict(st_vector).reshape(1,)
       
  return predictions[0] if predictions[0]>0 else 5

In [39]:
################################################## Function to get most similar texts to the user input #########################################################################

def get_multiple_similar_text_from_database(user_input_text, n,quotation_len_minimum_threshold ):
  global underscored_corpus
  global space_clean_corpus 
  user_input_dict,_= get_label_dict(user_input_text)

  
  output = {
      "space_seperated":[],
      "underscore_seperated":[] 
      }

  if "description" in user_input_dict.keys():
    user_input_dict["description"]=" ".join([x.strip() for x in user_input_dict["description"].split(',')])
    corrected_user_input = " ".join(user_input_dict["description"].split(" , "))
    for word in user_input_text.split():
        if word not in corrected_user_input:
            corrected_user_input+=" "+word
    user_input_text=corrected_user_input

  else:
    return output
  try:
      clean_text_vectorizer = TfidfVectorizer(ngram_range=(1,5))
      response = clean_text_vectorizer.fit_transform(list(space_clean_corpus))
  except: 
      print("Empty space clean vectorizer.Cannot fit_transform vectorizer on empty text corpus in the get_multiple_similar_text_from_database() function")
      return output

  input_text_vector = clean_text_vectorizer.transform([user_input_text])
  cosine_similarities = linear_kernel(input_text_vector, response).flatten()
  related_docs_indices = cosine_similarities.argsort()[:-1*(n+1):-1]
  clean_texts = list(space_clean_corpus[related_docs_indices])
  underscored_texts = list(underscored_corpus[related_docs_indices])
 
#   quotation_len_minimum_threshold=15
  while len(output["space_seperated"])==0 and quotation_len_minimum_threshold>0:
      output = {
      "space_seperated":[],
      "underscore_seperated":[] 
      }
      for i in range(len(clean_texts)):
        clean_text=re.sub('[^A-Za-z0-9_]+', ' ',clean_texts[i])
        underscored_text= re.sub('[^A-Za-z0-9_]+', ' ',underscored_texts[i])
        if len(set(user_input_dict["description"].split()).intersection(set(clean_text.split())))!=0 and len(underscored_text.split())>quotation_len_minimum_threshold :
            output["space_seperated"].append(clean_text.strip())
            output["underscore_seperated"].append(underscored_text.strip())
      quotation_len_minimum_threshold=quotation_len_minimum_threshold-1
  return output
  

In [40]:
get_multiple_similar_text_from_database("saree", 1, 1)['underscore_seperated']

[]

In [41]:
def predict_fabric(user_txt, quotation_len_minimum_threshold, batch_size):
    try:
        predicted_fabrics = False
        fabric_prediction_quotation_text="NONE"
        while not predicted_fabrics:
            if quotation_len_minimum_threshold>=50:
                return predicted_fabrics
            fabric_prediction_quotation_text = get_multiple_similar_text_from_database(user_txt,
                                                        batch_size, 
                                                        quotation_len_minimum_threshold)["underscore_seperated"][0]
            fabric_prediction_dict, _= get_label_dict(fabric_prediction_quotation_text)
            #print(fabric_prediction_quotation_text)
            if "fabric" in fabric_prediction_dict.keys():
                predicted_fabrics=fabric_prediction_dict["fabric"].split(" , ")
            quotation_len_minimum_threshold+=1

        incomplete_dict_from_user_input, undefined_word_present=get_label_dict(user_txt)
        if "fabric" not in incomplete_dict_from_user_input.keys():
            return predicted_fabrics,  fabric_prediction_quotation_text
        for fabric in incomplete_dict_from_user_input["fabric"].split(" , "):
            if fabric not in " ".join(predicted_fabrics):
                predicted_fabrics.append(fabric)
        return predicted_fabrics, fabric_prediction_quotation_text
    except:
        print("Empty database or some error in predict_fabric() function")
        return [],"empty"
        

In [42]:
fabrics, fabric_text=predict_fabric("nylon shirt embroidery and threads", quotation_len_minimum_threshold=3, batch_size=200)
print("fabrics predicted: ", fabrics)
print("Quotation text used : ",fabric_text)

Empty space clean vectorizer.Cannot fit_transform vectorizer on empty text corpus in the get_multiple_similar_text_from_database() function
Empty database or some error in predict_fabric() function
fabrics predicted:  []
Quotation text used :  empty


In [43]:
def predict_trims(user_txt, quotation_len_minimum_threshold, batch_size):
    try:
        predicted_trims = False
        trims_prediction_quotation_text="NONE"
        while not predicted_trims:
            if quotation_len_minimum_threshold>=50:
                return predicted_trims
            trims_prediction_quotation_text = get_multiple_similar_text_from_database(user_txt,
                                                        batch_size, 
                                                        quotation_len_minimum_threshold)["underscore_seperated"][0]
            trims_prediction_dict, _= get_label_dict(trims_prediction_quotation_text)
            #print(trims_prediction_quotation_text)
            if "trims" in trims_prediction_dict.keys():
                predicted_trims=trims_prediction_dict["trims"].split(" , ")
            quotation_len_minimum_threshold+=1

        incomplete_dict_from_user_input, undefined_word_present=get_label_dict(user_txt)
        if "trims" not in incomplete_dict_from_user_input.keys():
            return predicted_trims, trims_prediction_quotation_text
        for trim in incomplete_dict_from_user_input["trims"].split(" , "):
            if trim not in " ".join(predicted_trims):
                predicted_trims.append(trim)
        return predicted_trims, trims_prediction_quotation_text
    except:
        print("Empty database or some error in predict_fabric() function")
        return [],"empty"
        

In [44]:
trims, trim_text=predict_trims("silk blouse", quotation_len_minimum_threshold=8, batch_size=200)
print("trims predicted: ", trims)
print("Quotation text used : ",trim_text)


Empty space clean vectorizer.Cannot fit_transform vectorizer on empty text corpus in the get_multiple_similar_text_from_database() function
Empty database or some error in predict_fabric() function
trims predicted:  []
Quotation text used :  empty


In [45]:
def predict_cmt(user_txt, quotation_len_minimum_threshold, batch_size):
    try:
        predicted_cmt = False
        cmt_prediction_quotation_text="NONE"
        while not predicted_cmt:
            if quotation_len_minimum_threshold>=50:
                return predicted_cmt
            cmt_prediction_quotation_text = get_multiple_similar_text_from_database(user_txt,
                                                        batch_size, 
                                                        quotation_len_minimum_threshold)["underscore_seperated"][0]
            cmt_prediction_dict, _= get_label_dict(cmt_prediction_quotation_text)
            #print(cmt_prediction_quotation_text)
            if "cmt" in cmt_prediction_dict.keys():
                predicted_cmt=cmt_prediction_dict["cmt"].split(" , ")
            quotation_len_minimum_threshold+=1

        incomplete_dict_from_user_input, undefined_word_present=get_label_dict(user_txt)
        if "cmt" not in incomplete_dict_from_user_input.keys():
            return predicted_cmt, cmt_prediction_quotation_text
        for cmt in incomplete_dict_from_user_input["cmt"].split(" , "):
            if cmt not in " ".join(predicted_cmt):
                predicted_cmt.append(cmt)
        return predicted_cmt, cmt_prediction_quotation_text
    except:
        print("Empty database or some error in predict_fabric() function")
        return [],"empty"
        

In [46]:
cmt, cmt_text=predict_cmt("shirt", quotation_len_minimum_threshold=9, batch_size=200)
print("cmt predicted: ", cmt)
print("Quotation text used : ",cmt_text)
# print("Quotation_number/Order number: ",quotation_list[orignal_text_list.index(cmt_text)])

Empty space clean vectorizer.Cannot fit_transform vectorizer on empty text corpus in the get_multiple_similar_text_from_database() function
Empty database or some error in predict_fabric() function
cmt predicted:  []
Quotation text used :  empty


In [47]:
def predict_packing(user_txt, quotation_len_minimum_threshold, batch_size):
    try:
        predicted_packing = False
        packing_prediction_quotation_text="NONE"
        while not predicted_packing:
            if quotation_len_minimum_threshold>=50:
                return predicted_packing
            packing_prediction_quotation_text = get_multiple_similar_text_from_database(user_txt,
                                                        batch_size, 
                                                        quotation_len_minimum_threshold)["underscore_seperated"][0]
            packing_prediction_dict, _= get_label_dict(packing_prediction_quotation_text)
            #print(packing_prediction_quotation_text)
            if "packing" in packing_prediction_dict.keys():
                predicted_packing=packing_prediction_dict["packing"].split(" , ")
            quotation_len_minimum_threshold+=1

        incomplete_dict_from_user_input, undefined_word_present=get_label_dict(user_txt)
        if "packing" not in incomplete_dict_from_user_input.keys():
            return predicted_packing, packing_prediction_quotation_text
        for packing in incomplete_dict_from_user_input["packing"].split(" , "):
            if packing not in " ".join(predicted_packing):
                predicted_packing.append(packing)
        return predicted_packing, packing_prediction_quotation_text
    except:
        print("Empty database or some error in predict_fabric() function")
        return [],"empty"
        


In [48]:
packing, packing_text=predict_packing("silk blouse", quotation_len_minimum_threshold=15, batch_size=200)
print("packing predicted: ", packing)
print("Quotation text used : ",packing_text)
# print("Quotation_number/Order number: ",quotation_list[orignal_text_list.index(packing_text)])

Empty space clean vectorizer.Cannot fit_transform vectorizer on empty text corpus in the get_multiple_similar_text_from_database() function
Empty database or some error in predict_fabric() function
packing predicted:  []
Quotation text used :  empty


In [49]:

######################################################################### Version2 ##########################################################################################
def get_item_breakup_dictionary_v4(user_txt):
  
  user_txt=user_txt.split()
  for word in user_txt:
    if word not in label_classifier_vectorizer.vocabulary_.keys() and word not in new_label_classification_data["product_info"]:
        user_txt.remove(word)
  user_txt=" ".join(user_txt)

  incomplete_dict_from_user_input, undefined_word_present=get_label_dict(user_txt)

  if "description" not in incomplete_dict_from_user_input.keys():
    if undefined_word_present!=False:
      print ("Undeined entities: "+undefined_word_present)
      print("Not in database")
    return False
  
  incomplete_dict_from_user_input["description"]=" ".join([x.strip() for x in incomplete_dict_from_user_input["description"].split(',')])
  if set(incomplete_dict_from_user_input["description"].split()).intersection(set(list(label_classifier_vectorizer.vocabulary_.keys())))==0:
    print("Unknown description or new garment entered")
    return False
    
  user_txt=""
  for key in incomplete_dict_from_user_input.keys():
    user_txt  += " "+incomplete_dict_from_user_input[key]   #Modifying the user input to the values of incomplete_dic_from_user_input
    
  user_txt=user_txt.strip()

  
  batch_size=200
  
  most_likely_texts_dictionary=get_multiple_similar_text_from_database(user_txt, batch_size, quotation_len_minimum_threshold=8)
  most_likely_underscore_seperated_texts= most_likely_texts_dictionary["underscore_seperated"]
  most_likely_space_seperated_texts = most_likely_texts_dictionary["space_seperated"]
  space_clean_paragraph_words = (" ".join(most_likely_space_seperated_texts)).split()

  if len(most_likely_underscore_seperated_texts)==0:
        print("Nothing in database")
        return False
  
  output_dict=incomplete_dict_from_user_input 
  

  try:
      predicted_fabrics =predict_fabric(user_txt, quotation_len_minimum_threshold=8, batch_size=batch_size)[0]
      predicted_trims =predict_trims(user_txt, quotation_len_minimum_threshold=10, batch_size=batch_size)[0]
      predicted_cmt =predict_cmt(user_txt, quotation_len_minimum_threshold=9, batch_size=batch_size)[0]
      predicted_packing =predict_packing(user_txt, quotation_len_minimum_threshold=20, batch_size=batch_size)[0]
 
      predictions={"fabric":predicted_fabrics, "trims":predicted_trims, "cmt":predicted_cmt,"packing":predicted_packing}
  
      for key in predictions.keys():
        if not predictions[key]:
            continue
   
        output_dict[key]=" , ".join(predictions[key])
      
  except:
    most_matched_text_with_user_input=most_likely_texts_dictionary["underscore_seperated"][0].split()
    for key in incomplete_dict_from_user_input.keys():
        if incomplete_dict_from_user_input[key] not in most_matched_text_with_user_input:
            most_matched_text_with_user_input.append(incomplete_dict_from_user_input[key])
    most_matched_text_with_user_input=" ".join(most_matched_text_with_user_input)   
    output_dict,_=get_label_dict(most_matched_text_with_user_input)
  output_dict["description"]=" ".join(incomplete_dict_from_user_input["description"].split(" , "))
  
  return output_dict


In [50]:
get_item_breakup_dictionary_v4("saree")

False

In [51]:
get_label_dict("nylon shirt embroidery and threads")

({'fabric': 'nylon',
  'description': 'shirt',
  'cmt': 'embroidery',
  'trims': 'thread'},
 False)

In [52]:
# most_likely_texts_dictionary=get_multiple_similar_text_from_database("blouse",2, 200)
# most_likely_underscore_seperated_texts= most_likely_texts_dictionary["underscore_seperated"]
# current_dict, undefined_word_present = get_label_dict((most_likely_underscore_seperated_texts[0]))
# current_dict

In [53]:
def dummy_output_dict(user_input_text):
  item_breakup_dict, new_description=get_label_dict(user_input_text)
  if "description" in item_breakup_dict.keys() and new_description:
      item_breakup_dict["description"]=new_description + " "+ item_breakup_dict["description"]
  elif "description" not in item_breakup_dict.keys() and  new_description:
      item_breakup_dict["description"]= new_description
  elif "description" not in item_breakup_dict.keys() and  not new_description:
      item_breakup_dict["description"]="NA"
  #print(user_txt)
  if "fabric" not in item_breakup_dict.keys(): 
    item_breakup_dict["fabric"]="-"
  if "trims" not in item_breakup_dict.keys():
    item_breakup_dict["trims"]="-"
  if "cmt" not in item_breakup_dict.keys():
    item_breakup_dict["cmt"]="-"
  if "packing" not in item_breakup_dict.keys():
    item_breakup_dict["packing"]="-"   
    
  output_dict= item_breakup_dict
  for key in item_breakup_dict.keys():
    if key=="description":
        output_dict[key]=" ".join([x.strip() for x in item_breakup_dict[key].split(',')])
        continue
    if key=="fabric":
      
      used_fabrics=item_breakup_dict[key].split(',')
      output_dict[key]={}
      output_dict[key]["items"]=[]
      for fabric in used_fabrics:
        output_dict[key]["items"].append({"name":fabric.strip(),
                                          "avg": 0,
                                          "rate": 0,
                                          "amount": 0,
                                          "uom":""
                                          })
      output_dict[key]["total_cost"]=0
   
    elif key =="trims":
      
      items = item_breakup_dict[key].split(',')
      output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
       
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 0,
          "rate": 0,
          "amount": 0,
          "uom":""
        })
        
      output_dict[key]["total_cost"]=0
    elif key =="packing":
      items = item_breakup_dict[key].split(',')
      output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 0,
          "rate": 0,
          "amount": 0,
          "uom":""
        })
      output_dict[key]["total_cost"]=0
    else:
      items = item_breakup_dict[key].split(',')
      output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
       
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 0,
          "rate":0,
          "amount": 0,
          "uom":""
        })
      output_dict[key]["total_cost"]=0
  output_dict["Total cost"] = 0
  output_dict["buyer"] = ""
  return output_dict


In [54]:
def print_nested_dictionary(nested_dict):
  for key in nested_dict.keys():
    if type(nested_dict[key])==dict:
      print( key.upper()+" :")
      for k in nested_dict[key].keys():
        if type(nested_dict[key][k]==dict):
          print ("    "+k+" :")
          for i in nested_dict[key][k].keys():
            if type(nested_dict[key][k][i])==dict:
              print("       "+i + " : ")
              for j in nested_dict[key][k][i].keys():
                print("           "+j+": "+str(nested_dict[key][k][i][j]))
            else:
              print("       "+i+": "+str(nested_dict[key][k][i]))
        else:
          print("   "+k+" : "+nested_dict[key][k])
          print()
    else:
      print(key.upper()+" : "+str(nested_dict[key]))
    print()

In [55]:
def output_dict_with_cost_from_item_breakup_dic(nested_dict, buyer=""):
  item_breakup_dict = nested_dict
  print(nested_dict)
  if not item_breakup_dict:
    return False
  user_txt = item_breakup_dict["description"]
  #print(user_txt)
  total_cost=0
  output_dict={}
  buyer="" if len(buyer)<2 else buyer
  for key in item_breakup_dict.keys():
    if key=="fabric":
      total_fabric_quantity=0
      total_fabric_cost=0
      used_fabrics=item_breakup_dict[key].split(',')
      for fabric in used_fabrics:
        fabric_quantity = float(predict_fabric_quantity((buyer+" "+user_txt+ " "+fabric).strip() ))
        total_fabric_quantity+=fabric_quantity
        
      if "fabric" not in output_dict.keys():
        output_dict["fabric"]={}
      output_dict[key]["items"]=[]
      for fabric in used_fabrics:
        fabric_quantity = predict_fabric_quantity((buyer+" "+user_txt+ " "+fabric).strip())
        print((buyer+" "+user_txt+ " "+fabric).strip(), " HERE ", fabric_quantity)
        if total_fabric_quantity>2.5 and (buyer+" "+user_txt+ " "+fabric).strip() not in improved_item_avgs.keys():
            fabric_quantity= (fabric_quantity/total_fabric_quantity)*2.5
        fabric_price_rate = predict_fabric_rate((buyer+" "+user_txt+ " "+fabric).strip())
        print("Fabric rate for: ",buyer+" "+user_txt+ " "+fabric, " :", fabric_price_rate)
        
        total_cost+= fabric_quantity*fabric_price_rate
        total_fabric_cost+=fabric_quantity*fabric_price_rate
        output_dict[key]["items"].append({"name":fabric.strip(),
                                          "avg": str(round(fabric_quantity,2))+" meters",
                                          "rate": round(fabric_price_rate,3),
                                          "amount": round(fabric_quantity*fabric_price_rate,2),
                                          "uom": uom[fabric.strip()] if fabric.strip() in uom.keys() else ""
                                          })
      output_dict[key]["total_cost"]=round(total_fabric_cost,2)
    
    elif key =="description":
      output_dict[key]=" ".join([x.strip() for x in item_breakup_dict[key].split(',')])
    elif key =="buyer":
      output_dict[key]=item_breakup_dict[key].lower()
      continue
    elif key =="trims":
      
      items = item_breakup_dict[key].split(',')
      total_trim_cost=0
      if key not in output_dict.keys():
        output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        trim_quantity=float(predict_trim_quantity((buyer+" "+user_txt+ " "+item).strip()) )
        trim_price_rate=predict_trim_rate((buyer+" "+user_txt+ " "+item).strip())
         
        total_cost+= trim_quantity*trim_price_rate
        total_trim_cost+= trim_quantity*trim_price_rate
        if round(trim_quantity,2)==0:
            continue
        else:
            trim_quantity=round(trim_quantity,2)
            
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": round(trim_quantity,1),
          "rate": round(trim_price_rate,3),
          "amount": round(trim_quantity*trim_price_rate,2),
          "uom": uom[item.strip()] if item.strip() in uom.keys() else ""
        })
        
      output_dict[key]["total_cost"]=round(total_trim_cost,2)
    elif key =="packing":
      items = item_breakup_dict[key].split(',')
      total_packing_cost=0
      if key not in output_dict.keys():
        output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        price_rate=float(predict_trim_rate(user_txt+" "+item))
        total_cost+= price_rate
        total_packing_cost+=price_rate
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 1,
          "rate": round(price_rate,3),
          "amount": round(price_rate,2),
          "uom": uom[item.strip()] if item.strip() in uom.keys() else ""
        })
      output_dict[key]["total_cost"]=round(total_packing_cost,2)
    else:
      items = item_breakup_dict[key].split(',')
      total_cmt_cost=0
      if key not in output_dict.keys():
        output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        process_rate= float(predict_process_rate((buyer+" "+user_txt+ " "+item).strip()))
        total_cost+=process_rate
        total_cmt_cost+=process_rate
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 1,
          "rate": round(process_rate,3),
          "amount": round(process_rate,2),
          "uom": uom[item.strip()] if item.strip() in uom.keys() else ""
        })
      output_dict[key]["total_cost"]=round(total_cmt_cost)
  output_dict["Total cost"] = round(total_cost,3)
  return output_dict

In [56]:
buyer="abc"
buyer = buyer if len(buyer)>3 else "pqr"
buyer

'pqr'

In [57]:
def output_dict_with_cost(user_txt):
  item_breakup_dict = get_item_breakup_dictionary_v4(user_txt)
  
  if not item_breakup_dict:
    print("No valid breakup could be made from user text")
    return False

  if "buyer" not in item_breakup_dict.keys():
        item_breakup_dict["buyer"]=""
        
  user_txt = item_breakup_dict["description"]
  #print(user_txt)
  total_cost=0
  output_dict={}
  for key in item_breakup_dict.keys():
    if key=="fabric":
      total_fabric_quantity=0
      total_fabric_cost=0
      used_fabrics=item_breakup_dict[key].split(',')
      for fabric in used_fabrics:
        fabric_quantity = float(predict_fabric_quantity((item_breakup_dict["buyer"]+" "+user_txt+ " "+fabric).strip()))
        print((item_breakup_dict["buyer"]+" "+user_txt+ " "+fabric).strip(), "HERE ", fabric_quantity)
        total_fabric_quantity+=fabric_quantity
        
      if "fabric" not in output_dict.keys():
        output_dict["fabric"]={}
      output_dict[key]["items"]=[]
      for fabric in used_fabrics:
        fabric_quantity = float(predict_fabric_quantity((item_breakup_dict["buyer"]+" "+user_txt+ " "+fabric).strip()))
        if total_fabric_quantity>2.5 and (item_breakup_dict["buyer"]+" "+user_txt+ " "+fabric).strip() not in improved_item_avgs.keys():
            fabric_quantity= (fabric_quantity/total_fabric_quantity)*2.5
        fabric_price_rate = predict_fabric_rate(user_txt+" "+fabric.strip() )
        
        total_cost+= fabric_quantity*fabric_price_rate
        total_fabric_cost+=fabric_quantity*fabric_price_rate
        output_dict[key]["items"].append({"name":fabric.strip(),
                                          "avg": str(round(fabric_quantity,2))+" meters",
                                          "rate": round(fabric_price_rate,3),
                                          "amount": round(fabric_quantity*fabric_price_rate,2),
                                          "uom": uom[fabric.strip()] if fabric.strip() in uom.keys() else ""
                                          })
      output_dict[key]["total_cost"]=round(total_fabric_cost,2)
    
    elif key =="description":
      output_dict[key]=" ".join([x.strip() for x in item_breakup_dict[key].split(',')])
    elif key =="buyer":
        output_dict[key]=item_breakup_dict[key].lower().strip()
      
    elif key =="trims":
      
      items = item_breakup_dict[key].split(',')
      total_trim_cost=0
    
      if key not in output_dict.keys():
        output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        trim_quantity=float(predict_trim_quantity((item_breakup_dict["buyer"]+" "+user_txt+" "+item).strip()) )
        trim_price_rate=predict_trim_rate((item_breakup_dict["buyer"]+" "+user_txt+" "+item).strip())
       
        total_cost+= trim_quantity*trim_price_rate
        total_trim_cost+= trim_quantity*trim_price_rate
        if round(trim_quantity,2)==0:
            continue
        else:
            trim_quantity=round(trim_quantity,2)
            
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": round(trim_quantity,1),
          "rate": round(trim_price_rate,3),
          "amount": round(trim_quantity*trim_price_rate,2),
          "uom": uom[item.strip()] if item.strip() in uom.keys() else ""
        })
        
      output_dict[key]["total_cost"]=round(total_trim_cost,2)
    elif key =="packing":
      items = item_breakup_dict[key].split(',')
      total_packing_cost=0
      if key not in output_dict.keys():
        output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        price_rate=float(predict_trim_rate((item_breakup_dict["buyer"]+" "+user_txt+" "+item).strip()))
        total_cost+= price_rate
        total_packing_cost+=price_rate
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 1,
          "rate": round(price_rate,3),
          "amount": round(price_rate,2),
          "uom": uom[item.strip()] if item.strip() in uom.keys() else ""
        })
      output_dict[key]["total_cost"]=round(total_packing_cost,2)
    else:
      items = item_breakup_dict[key].split(',')
      total_cmt_cost=0
      if key not in output_dict.keys():
        output_dict[key]={}
      output_dict[key]["items"]=[]
      for item in list(set(items)):
        item= item.strip()
        process_rate= float(predict_process_rate((item_breakup_dict["buyer"]+" "+user_txt+" "+item).strip()))
        total_cost+=process_rate
        total_cmt_cost+=process_rate
        output_dict[key]["items"].append({
          "name":item.strip(),
          "avg": 1,
          "rate": round(process_rate,3),
          "amount": round(process_rate,2),
          "uom": uom[item.strip()] if item.strip() in uom.keys() else ""
        })
      output_dict[key]["total_cost"]=round(total_cmt_cost)
  output_dict["Total cost"] = round(total_cost,3)
  return output_dict

In [58]:
predict_fabric_quantity("saree silk")

1.1599894

In [59]:
output_dict_with_cost("saree")

No valid breakup could be made from user text


False

In [60]:
def return_print_output(nested_dict):
    output="======================================================================================================<br />"
    keys= ["description", "fabric", "cmt", "trims", "packing", "total cost"]
    for key in keys:
        if key not in nested_dict.keys():
            continue
        if type(nested_dict[key])==dict:
          output+=( key.upper()+" :")+' <br />'
          for k in nested_dict[key].keys():
            if type(nested_dict[key][k])==dict:
              output+=("&nbsp&nbsp&nbsp&nbsp"+k+" :")+ ' <br />'
              for i in nested_dict[key][k].keys():
                if type(nested_dict[key][k][i])==dict:
                  output+=("&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp"+i + " : ")+' <br />'
                  for j in nested_dict[key][k][i].keys():
                    output+=("&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp"+j+": "+str(nested_dict[key][k][i][j]))+'<br />'
                else:
                  output+=("&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp"+i+": "+str(nested_dict[key][k][i]))+ '<br />'
            else:
              output+= ' <br />--------------------------------------------------------------------------------------- <br />'
              output+=("&nbsp&nbsp&nbsp&nbsp"+str(k)+" : "+str(nested_dict[key][k]))+ ' <br />'
              output+= ' <br />--------------------------------------------------------------------------------------- <br />'
        else:
          output+=(key.upper()+" : "+str(nested_dict[key]))+' <br />'
        output+= ' <br />'
        output+="=====================================================================================================<br />"
    return output


In [ ]:
import sys

from flask import*
from flask_ngrok import run_with_ngrok
from flask_login import LoginManager, UserMixin, login_required, login_user, logout_user 
from openpyxl import load_workbook

from flask_login import LoginManager, UserMixin, login_required, login_user, logout_user 
app = Flask(__name__)
app.secret_key = "bluekaktus_secret"

# flask-login
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

global users

run_with_ngrok(app)
# silly user model
class User(UserMixin):
    pass

@login_manager.user_loader
def user_loader(email):
    if email not in users:
        return

    user = User()
    user.id = email
    return user

@app.route('/success/<string_query>',methods = ['POST', 'GET']) 
def success(string_query): 
    if request.method == 'POST': 
        return redirect(url_for('improve'), user_text_description=string_query) #user_text_description is argument of 'improve'
    
    user_specific_item_breakup_database={}
    if session["username"] in improved_item_breakup_database.keys():
        user_specific_item_breakup_database=improved_item_breakup_database[session["username"]]
    else:
        print("First entry by ", session["username"].upper())
        
        
    nested_dict={}
    cache=session["cache"]
########################## If the exact key is already avalable in the database where user is entering improved records########
    if string_query in user_specific_item_breakup_database.keys():
        item_breakup_text=user_specific_item_breakup_database[string_query]["underscore_seperated"]
        buyer=user_specific_item_breakup_database[string_query]["buyer"]

        print("FROM IMPROVED DATABASE used the breakup text corresponding to the key ",string_query, "-with breakup text: ",item_breakup_text)
        item_breakup_dict,_=get_label_dict(item_breakup_text)
        nested_dict=output_dict_with_cost_from_item_breakup_dic(item_breakup_dict, buyer=buyer)
        nested_dict["buyer"]=buyer
        
########################## If somewhat similar key is already avalable in the database where user is entering improved records########        
    elif most_similar_key_for_text_from_dict(string_query,user_specific_item_breakup_database):
        most_similar_key=most_similar_key_for_text_from_dict(string_query,user_specific_item_breakup_database)
        item_breakup_text=user_specific_item_breakup_database[most_similar_key]["underscore_seperated"]
        buyer=user_specific_item_breakup_database[most_similar_key]["buyer"]
        print("FROM IMPROVED DATABASE used the breakup text corresponding to the key ",most_similar_key, "-with breakup text: ",item_breakup_text)
        
        ############# Adding additional user inputted items into the string queried from imroved_item_breakup_database#######
        initial_len=len(item_breakup_text.split())
        user_query_dict,_=get_label_dict(string_query)
        for key in user_query_dict.keys():
            if key=="description":
                continue
            values= ["_".join(x.strip().split()) for x in user_query_dict[key].split(',')]
            item_breakup_text=" ".join(list(set(item_breakup_text.split()+values)))
        if len(item_breakup_text.split())!=initial_len:
            print("ADDED ADDITIONAL USER INPUT INFO FROM USER INPUT: ",user_query_dict)

   
        item_breakup_dict,_=get_label_dict(item_breakup_text)
        nested_dict=output_dict_with_cost_from_item_breakup_dic(item_breakup_dict, buyer=buyer) 
        nested_dict["buyer"]=buyer
#############################################################################################################################       
    elif string_query in cache.keys() and cache[string_query]["user"]==session["username"]:
        nested_dict=cache[string_query]
        
    else:
#         nested_dict= output_dict_with_cost(str(string_query))
#         print("used function: output_dict_with_cost() ",nested_dict)
        nested_dict=False
        
################################ If user inputs an entirely new garment #####################################################
    if not nested_dict:
        nested_dict=dummy_output_dict(string_query)
        session["cache"]=cache
        session["nested_dict"]=nested_dict
        print("SESSONS DICT: ", session["nested_dict"])
        return render_template('result.html', result=nested_dict, garment_text_description=string_query+"(new entry)")
##############################################################################################################################
    dummy_dict=dummy_output_dict(string_query)
    
    if "fabric" not in nested_dict.keys(): 
        nested_dict["fabric"]=dummy_dict["fabric"]
    if "trims" not in nested_dict.keys():
        nested_dict["trims"]=dummy_dict["trims"]
    if "cmt" not in nested_dict.keys():
        nested_dict["cmt"]=dummy_dict["cmt"]
    if "packing" not in nested_dict.keys():
        nested_dict["packing"]=dummy_dict["packing"] 
    if "buyer" not in nested_dict.keys():
        nested_dict["buyer"]="general"
    
    cache[string_query]=nested_dict 
    session["cache"]=cache
    session["nested_dict"]=nested_dict
    print("SESSONS DICT: ", session["nested_dict"])
    return render_template('result.html', result=nested_dict, garment_text_description=string_query)
    #garment_text_description is the variable of result.html 

@app.route('/success/improve/<user_text_description>',methods = ['POST', 'GET']) 
def improve(user_text_description): 
    
#     if request.method == 'POST':                                  #Handled by javascript fetch() function
#         return redirect(url_for('improved'))
    nested_dict=session["nested_dict"]
 
    return render_template('improve_with_bootstrap.html', result=nested_dict, garment_text_description=user_text_description)
    #garment_text_description is a variable of the improve.html file

@app.route('/improved',methods = ['POST','GET']) 
def improved(): 
    
    global improved_item_rates
    global improved_item_avgs
    global new_label_classification_data
    global improved_item_breakup_database
    global space_clean_corpus
    global underscored_corpus
    global uom
    
    improved_dict=request.get_json()['dtData']
    
    item_breakup_texts=item_beakup_text_from_dict(improved_dict)
    login_info=session["username"]
    improved_dict["user"]=login_info.lower()
    session["improved_dict"]=improved_dict
   
    
    ############################### If user enetered dashes without_values###################################################
    for key in improved_dict.keys():
        if key in ["description", "Total cost", "buyer", "user"]:
            continue
        for item in improved_dict[key]["items"]:
            if '-' in [item["name"],item["avg"], item["avg"]]:
                improved_dict[key]["items"].remove(item)
                
  ##########################################################################################################################
   
    
  ######A new garment information  entered by user stored. Buyer is stored seperately in the text ######################### 
    new_breakup_text_dict=item_beakup_text_from_dict(improved_dict)
    buyer=new_breakup_text_dict["buyer"]
    
    user_garment_text_description=session["user_garment_text_description"]
    #improved_item_breakup_database[user_garment_text_description]=new_breakup_text_dict
    if session["username"] not in improved_item_breakup_database.keys():
        improved_item_breakup_database[session["username"]]={}
    improved_item_breakup_database[session["username"]][user_garment_text_description]=new_breakup_text_dict
    ########################################################################################################################
    
    underscored_corpus=list(underscored_corpus)
    underscored_corpus.append((new_breakup_text_dict["underscore_seperated"]+" "+new_breakup_text_dict["buyer"]).strip())
    space_clean_corpus=list(space_clean_corpus)
    space_clean_corpus.append((new_breakup_text_dict["space_seperated"]+" "+new_breakup_text_dict["buyer"]).strip())                                   
    underscored_corpus=np.array(underscored_corpus)
    space_clean_corpus=np.array(space_clean_corpus)
    
    
    improved_item_rates=merge_dictionaries(current_dictionary=item_rates_from_dict(improved_dict), 
                                           database_dictionary=improved_item_rates)
    improved_item_avgs=merge_dictionaries(current_dictionary=item_avgs_from_dict(improved_dict), 
                                           database_dictionary=improved_item_avgs)
    #try:
    uom= merge_dictionaries(current_dictionary=uom_from_dict(improved_dict), 
                                               database_dictionary=uom)
    #except:
        #print("UOM not added")
    
    new_item_label_dict=item_labels_from_dict(improved_dict)
    ################################### Variables/arrays of texts used previously by these names#############################


          
    ######## Finding new user entered items/products/terms which were not in database previously and 
    ###########################################################adding them to the new_label_classification_dictionary##########
    for i in range(len(new_item_label_dict["product_info"])):
        #if new_item_label_dict["product_info"][i] not in label_classifier_vectorizer.vocabulary_.keys():
        if new_item_label_dict["product_info"][i] not in new_label_classification_data["product_info"]:
            new_label_classification_data["product_info"].append(new_item_label_dict["product_info"][i])
            new_label_classification_data["label"].append(new_item_label_dict["label"][i])
    ###########################################################################################################################
    
    ##########################putting the improved in mongo database collection named  user_improved_costing_collection #######
    improved_dict["user_garment_text_description"]=user_garment_text_description
    improved_dict["Total cost"]=float((improved_dict["Total cost"].strip()).split()[-1])
    improved_dict["username"]=session["username"]
    user_improved_costing_collection.insert_one(improved_dict)
    ##########################################################################################################################
    print("SESSION: ", session)
    return "Error if not string" 


@app.route('/', methods=['GET', 'POST'])
def login():
    login_dict={}
    if request.method=="POST":
        global users
        users=list(client["costing_engine_db_with_login"]["users"].find({"database_name":"username_password"}))[0]
        login_dict=request.form.to_dict()

        if login_dict["email"] in users.keys() and login_dict['password'] == users[login_dict["email"]]['password']:
            print(login_dict)
            email = login_dict['email']
            user = User()
            user.id = email
            login_user(user)
            session["username"]=email
            session["cache"]={}
            session["nested_dict"]={}
            return redirect(url_for('homepage'))
        
        return render_template("login.html", error_message="Incorrect credentials")

 
    return render_template("login.html")

    
  
@app.route('/logout')
def logout():
    session.clear()
    logout_user()
    return render_template("login.html", error_message="Logged out!")

@app.route('/home', methods=['POST', 'GET'])
@login_required
def homepage():
    return render_template("index.html")
    



@app.route('/user_input',methods = ['POST', 'GET']) 
def string_query_entry():  
    print("SESSION "+ str(session))
    if request.method == 'POST': 
        user = request.form['string_query'] 
        session["user_garment_text_description"]=user
        
        return redirect(url_for('success',string_query = user)) #string_query is a variable of the html file rendered in function 'success'
    else: 
        user = request.args.get('string_query') 
        return redirect(url_for('success',string_query = user)) 

if __name__ == '__main__': 
    app.run() 

I0924 12:41:56.655454  5444 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6ab9d490.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


I0924 12:42:05.273142  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:42:05] "GET / HTTP/1.1" 200 -
I0924 12:42:06.119115  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:42:06] "GET /static/2_blur.jpg HTTP/1.1" 404 -
I0924 12:42:11.027162  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:42:11] "POST / HTTP/1.1" 302 -


{'email': 'satyam', 'password': 'abc'}


I0924 12:42:11.868487  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:42:11] "GET /home HTTP/1.1" 200 -
I0924 12:42:15.761837  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:42:15] "POST /user_input HTTP/1.1" 302 -


...

{'email': 'pranjal', 'password': 'xyz'}


I0924 12:43:49.946825  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:43:49] "GET /home HTTP/1.1" 200 -
I0924 12:43:57.336622  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:43:57] "POST /user_input HTTP/1.1" 302 -


SESSION <SecureCookieSession {'_fresh': True, '_id': '5e7e9b19d5809fc5ec99a420ea29db1fdd874d5f60117883cfe536d119412ce0a6f74eb8f9b19281cc2fc9740ba4e8a81add738b8a078bee8c7c6663d236a13a', 'cache': {}, 'nested_dict': {}, 'user_id': 'pranjal', 'username': 'pranjal'}>


I0924 12:43:58.139011  5444 _internal.py:122] 127.0.0.1 - - [24/Sep/2019 12:43:58] "GET /success/shirt HTTP/1.1" 200 -


FROM IMPROVED DATABASE used the breakup text corresponding to the key  shirt -with breakup text:  shirt
{'description': 'shirt'}
SESSONS DICT:  {'description': 'shirt', 'Total cost': 0, 'buyer': 'mango', 'fabric': {'items': [{'name': '-', 'avg': 0, 'rate': 0, 'amount': 0, 'uom': ''}], 'total_cost': 0}, 'trims': {'items': [{'name': '-', 'avg': 0, 'rate': 0, 'amount': 0, 'uom': ''}], 'total_cost': 0}, 'cmt': {'items': [{'name': '-', 'avg': 0, 'rate': 0, 'amount': 0, 'uom': ''}], 'total_cost': 0}, 'packing': {'items': [{'name': '-', 'avg': 0, 'rate': 0, 'amount': 0, 'uom': ''}], 'total_cost': 0}}


In [66]:
improved_item_breakup_database

{}